In [1]:
#!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git
#!pip install wordcloud

In [2]:
import pandas as pd
import csv
import re
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from wordcloud import WordCloud
import nltk 
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer() #initializing the class or rather creating an instance of the class
from nltk.corpus import stopwords

pd.set_option('display.max_colwidth', None)


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oo_wa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Creating list to append tweet data to
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('carjacking south africa since:2010-01-01 until:2022-01-01').get_items()):
    if i>2000:
        break
    attributes_container.append([tweet.user.username, tweet.date, tweet.likeCount, tweet.sourceLabel, tweet.content])
    
# Creating a dataframe to load the list
tweets_df = pd.DataFrame(attributes_container, columns=["User", "Date Created", "Number of Likes", "Source of Tweet", "Tweet"])

C:\Users\oo_wa\AppData\Local\Temp\ipykernel_20440\101331613.py:11: FutureWarning: content is deprecated, use rawContent instead
  attributes_container.append([tweet.user.username, tweet.date, tweet.likeCount, tweet.sourceLabel, tweet.content])


In [5]:
tweets_df.head()

,User,Date Created,Number of Likes,Source of Tweet,Tweet
0,MigrantWatchRSA,2021-12-17 15:21:12+00:00,0,Twitter for Android,#SouthAfrica #xenophobia #africa #mozambique #zimbabwe #carjacking
1,c4i,2021-12-16 03:22:28+00:00,0,Twitter for iPhone,"@ECCSC_chicago @nbcchicago @AldTomTunney @AldermanSmith43 @Chicago_Police In South Africa, a anti-carjacking device that acts like a flamethrower was made legal to use in self-defense of a violent carjacking and I won’t be the least bit surprised if someone starts importing these to the U.S. \n\nhttps://t.co/Vc6RBi4DLW"
2,bellisaurius,2021-12-07 04:16:06+00:00,1,Twitter Web App,"@Beowulf_i_am @FenixAmmunition South Africa leads the way on carjacking defense, I think. https://t.co/iPB98bRxAI"
3,IJSanders,2021-11-17 16:46:03+00:00,1,Twitter Web App,"@TheMikeChase I must say, I remember someone in South Africa had flamethrowers on their car a couple decades ago. I think they came out the sides and were an anti-carjacking device."
4,ArmouredM,2021-11-13 02:43:47+00:00,2,Twitter for Android,Be prepared should you become a target.\n\nContact us today to ARMOUR your vehicle.\n\nSecure Your Drive. Secure your Life.\nI AM ARMOURED MOBILITY.\n\n#carjacking #hijack #kidnapping #SouthAfrica #armour #protection #security #safety\n\nhttps://t.co/5LhGZqikpn


In [6]:
tweets_df.shape

(1541, 5)

In [7]:
data_df=tweets_df.Tweet

In [8]:
data_df

0                                                                                                                                                                                                                                                                      #SouthAfrica #xenophobia #africa #mozambique #zimbabwe #carjacking
1       @ECCSC_chicago @nbcchicago @AldTomTunney @AldermanSmith43 @Chicago_Police In South Africa, a anti-carjacking device that acts like a flamethrower was made legal to use in self-defense of a violent carjacking and I won’t be the least bit surprised if someone starts importing these to the U.S.  \n\nhttps://t.co/Vc6RBi4DLW
2                                                                                                                                                                                                                       @Beowulf_i_am @FenixAmmunition South Africa leads the way on carjacking defense, I think. https://t.co/iPB98bRxAI
3         

# Data Cleaning

In [9]:
import re
import string
from cleantext import clean

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = clean(text, no_emoji=True)

    return text

round1 = lambda x: clean_text_round1(x)

In [10]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.apply(round1))
data_clean

,Tweet
0,southafrica xenophobia africa mozambique zimbabwe carjacking
1,eccscchicago nbcchicago aldtomtunney chicagopolice in south africa a anticarjacking device that acts like a flamethrower was made legal to use in selfdefense of a violent carjacking and i won't be the least bit surprised if someone starts importing these to the us
2,beowulfiam fenixammunition south africa leads the way on carjacking defense i think
3,themikechase i must say i remember someone in south africa had flamethrowers on their car a couple decades ago i think they came out the sides and were an anticarjacking device
4,be prepared should you become a target\ncontact us today to armour your vehicle\nsecure your drive secure your life\ni am armoured mobility\ncarjacking hijack kidnapping southafrica armour protection security safety
...,...
1536,carjacking in johannesburg httpbitlychcftg southafrica
1537,not content with a mammoth soccer world cup debt south africa now wants the olympics would carjacking be a sport
1538,harftimesen some tourism highlights for south africa carjacking hippo attacks and aids museum all worth a look
1539,i now know why i wont visit south africa murder carjacking rape running red lights no constant use of trumpets at football match


# Corpus
We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.



In [11]:
data_df

0                                                                                                                                                                                                                                                                      #SouthAfrica #xenophobia #africa #mozambique #zimbabwe #carjacking
1       @ECCSC_chicago @nbcchicago @AldTomTunney @AldermanSmith43 @Chicago_Police In South Africa, a anti-carjacking device that acts like a flamethrower was made legal to use in self-defense of a violent carjacking and I won’t be the least bit surprised if someone starts importing these to the U.S.  \n\nhttps://t.co/Vc6RBi4DLW
2                                                                                                                                                                                                                       @Beowulf_i_am @FenixAmmunition South Africa leads the way on carjacking defense, I think. https://t.co/iPB98bRxAI
3         

In [12]:
# Let's pickle it for later use
data_df.to_pickle("corpus.pkl")

# Document-Term Matrix
For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [13]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.Tweet)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

C:\Users\oo_wa\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abalone,abbymartin,abc,abcnews,able,abramjee,abused,abuwankinobbi,acceptable,accessibility,...,zahriyeh,zaksam,zarebel,zealand,zimbabwe,zimbabwean,zone,zoologeek,zuidafrika,zuma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [16]:
import pickle
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

# Topic Modeling

In [24]:
data = pd.read_pickle('dtm.pkl')
data

,abalone,abbymartin,abc,abcnews,able,abramjee,abused,abuwankinobbi,acceptable,accessibility,...,zahriyeh,zaksam,zarebel,zealand,zimbabwe,zimbabwean,zone,zoologeek,zuidafrika,zuma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
from gensim import matutils, models
import scipy.sparse

In [26]:
tdm = data.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,...,1531,1532,1533,1534,1535,1536,1537,1538,1539,1540
abalone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abbymartin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abcnews,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [28]:
cv = pickle.load(open("cv.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [29]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.089*"south" + 0.088*"africa" + 0.087*"carjacking" + 0.039*"boy" + 0.037*"dies" + 0.014*"katie" + 0.013*"victim" + 0.013*"price" + 0.009*"dragged" + 0.006*"vehicle"'),
 (1,
  '0.067*"carjacking" + 0.061*"south" + 0.057*"africa" + 0.018*"british" + 0.017*"newlywed" + 0.015*"southafrica" + 0.015*"honeymoon" + 0.010*"dead" + 0.010*"dewani" + 0.008*"anni"')]

In [30]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.069*"carjacking" + 0.061*"south" + 0.059*"africa" + 0.030*"british" + 0.029*"newlywed" + 0.023*"honeymoon" + 0.019*"attempted" + 0.017*"dewani" + 0.016*"dead" + 0.014*"anni"'),
 (1,
  '0.074*"carjacking" + 0.070*"south" + 0.062*"africa" + 0.023*"victim" + 0.021*"twitter" + 0.016*"helps" + 0.013*"tweet" + 0.013*"saves" + 0.012*"new" + 0.012*"used"'),
 (2,
  '0.089*"south" + 0.089*"africa" + 0.086*"carjacking" + 0.043*"boy" + 0.041*"dies" + 0.015*"katie" + 0.014*"price" + 0.010*"dragged" + 0.008*"murder" + 0.006*"vehicle"')]

In [31]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.079*"south" + 0.078*"africa" + 0.077*"carjacking" + 0.031*"katie" + 0.029*"victim" + 0.029*"price" + 0.014*"attempted" + 0.013*"tweet" + 0.013*"lucky" + 0.013*"saves"'),
 (1,
  '0.064*"carjacking" + 0.043*"south" + 0.042*"africa" + 0.027*"southafrica" + 0.010*"killed" + 0.009*"crime" + 0.008*"flamethrowers" + 0.008*"news" + 0.006*"murdered" + 0.006*"reveals"'),
 (2,
  '0.119*"boy" + 0.113*"dies" + 0.091*"south" + 0.085*"africa" + 0.082*"carjacking" + 0.027*"dragged" + 0.017*"stolen" + 0.016*"vehicle" + 0.005*"police" + 0.004*"africas"'),
 (3,
  '0.086*"carjacking" + 0.086*"south" + 0.084*"africa" + 0.020*"british" + 0.019*"newlywed" + 0.016*"honeymoon" + 0.015*"dewani" + 0.012*"dead" + 0.010*"new" + 0.009*"anni"')]

# Topic Modeling- Nouns Only

In [32]:
# a function to pull out nouns from a string of text

from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [33]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,Tweet
0,southafrica xenophobia africa mozambique zimbabwe carjacking
1,eccscchicago nbcchicago aldtomtunney chicagopolice in south africa a anticarjacking device that acts like a flamethrower was made legal to use in selfdefense of a violent carjacking and i won't be the least bit surprised if someone starts importing these to the us
2,beowulfiam fenixammunition south africa leads the way on carjacking defense i think
3,themikechase i must say i remember someone in south africa had flamethrowers on their car a couple decades ago i think they came out the sides and were an anticarjacking device
4,be prepared should you become a target\ncontact us today to armour your vehicle\nsecure your drive secure your life\ni am armoured mobility\ncarjacking hijack kidnapping southafrica armour protection security safety
...,...
1536,carjacking in johannesburg httpbitlychcftg southafrica
1537,not content with a mammoth soccer world cup debt south africa now wants the olympics would carjacking be a sport
1538,harftimesen some tourism highlights for south africa carjacking hippo attacks and aids museum all worth a look
1539,i now know why i wont visit south africa murder carjacking rape running red lights no constant use of trumpets at football match


In [36]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\oo_wa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [37]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.Tweet.apply(nouns))
data_nouns

,Tweet
0,southafrica xenophobia zimbabwe
1,aldtomtunney chicagopolice device flamethrower selfdefense carjacking i bit someone
2,beowulfiam fenixammunition south way defense i
3,themikechase i someone africa flamethrowers car couple decades sides device
4,target contact today vehicle secure life i mobility hijack kidnapping protection security safety
...,...
1536,johannesburg httpbitlychcftg southafrica
1537,soccer world cup debt south africa olympics sport
1538,tourism highlights hippo attacks aids museum look
1539,wont visit south murder rape lights use trumpets football match


In [42]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['south','africa','southafrica','carjacking','twitter','anni','dewani','price','katie','honeymoon']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.Tweet)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

C:\Users\oo_wa\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abalone,abbymartin,abc,abcnews,abramjee,abuwankinobbi,accidents,ace,action,activity,...,yvonnemaphosa,zack,zagahfresh,zaksam,zarebel,zealand,zimbabwe,zone,zoologeek,zuma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# Create the gensim corpus
corpusn= matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn= dict((v, k) for k, v in cvn.vocabulary_.items())

In [50]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.021*"murder" + 0.015*"crime" + 0.014*"women" + 0.013*"motorists" + 0.012*"method" + 0.012*"photos" + 0.009*"trial" + 0.009*"shrien" + 0.008*"kids" + 0.007*"lucky"'),
 (1,
  '0.093*"boy" + 0.071*"dies" + 0.031*"victim" + 0.019*"news" + 0.015*"vehicle" + 0.014*"world" + 0.013*"wheelchair" + 0.013*"tennis" + 0.013*"saves" + 0.009*"car"')]

In [53]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.053*"victim" + 0.022*"saves" + 0.020*"women" + 0.018*"motorists" + 0.018*"method" + 0.017*"murder" + 0.017*"photos" + 0.016*"wheelchair" + 0.015*"tennis" + 0.015*"girlfriends"'),
 (1,
  '0.125*"boy" + 0.096*"dies" + 0.019*"world" + 0.016*"vehicle" + 0.012*"crime" + 0.011*"news" + 0.011*"rate" + 0.011*"man" + 0.008*"blis" + 0.008*"johannesburg"'),
 (2,
  '0.019*"car" + 0.019*"news" + 0.018*"children" + 0.014*"murder" + 0.011*"flamethrowers" + 0.011*"tennis" + 0.011*"wheelchair" + 0.011*"champ" + 0.010*"police" + 0.010*"woman"')]

In [54]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.058*"victim" + 0.030*"saves" + 0.027*"women" + 0.024*"motorists" + 0.023*"method" + 0.022*"photos" + 0.020*"girlfriends" + 0.012*"huffingtonpost" + 0.009*"flamethrowers" + 0.009*"tweet"'),
 (1,
  '0.021*"news" + 0.014*"boy" + 0.013*"lucky" + 0.013*"couple" + 0.012*"cape" + 0.012*"police" + 0.012*"car" + 0.011*"suspect" + 0.011*"town" + 0.011*"friend"'),
 (2,
  '0.031*"man" + 0.022*"car" + 0.019*"blis" + 0.017*"flamethrowers" + 0.012*"throwers" + 0.012*"family" + 0.012*"cars" + 0.011*"flame" + 0.010*"incident" + 0.009*"problem"'),
 (3,
  '0.133*"boy" + 0.106*"dies" + 0.022*"wheelchair" + 0.021*"tennis" + 0.021*"world" + 0.019*"murder" + 0.018*"vehicle" + 0.014*"news" + 0.011*"champion" + 0.011*"trial"')]

# Topic Modeling- Nouns and Adjectives

In [55]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [57]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.Tweet.apply(nouns_adj))
data_nouns_adj

,Tweet
0,southafrica xenophobia mozambique zimbabwe
1,nbcchicago aldtomtunney chicagopolice south anticarjacking device flamethrower legal selfdefense violent carjacking i least bit surprised someone
2,beowulfiam fenixammunition south africa way defense i
3,themikechase i i someone south africa flamethrowers car couple decades i sides anticarjacking device
4,target contact today vehicle drive secure life i mobility hijack kidnapping armour protection security safety
...,...
1536,johannesburg httpbitlychcftg southafrica
1537,content mammoth soccer world cup debt south africa olympics sport
1538,tourism highlights south africa hippo attacks aids museum look
1539,i wont visit south africa murder rape red lights constant use trumpets football match


In [59]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.Tweet)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

C:\Users\oo_wa\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abalone,abbymartin,abc,abcnews,able,abramjee,abuwankinobbi,acceptable,accidents,ace,...,zack,zagahfresh,zaksam,zarebel,zealand,zimbabwe,zimbabwean,zone,zoologeek,zuma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [61]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.024*"british" + 0.016*"attempted" + 0.014*"new" + 0.013*"wheelchair" + 0.013*"tennis" + 0.013*"dead" + 0.010*"women" + 0.010*"crime" + 0.009*"motorists" + 0.009*"man"'),
 (1,
  '0.078*"boy" + 0.059*"dies" + 0.021*"victim" + 0.012*"vehicle" + 0.012*"lucky" + 0.011*"saves" + 0.010*"news" + 0.010*"world" + 0.010*"alive" + 0.008*"car"')]

In [62]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.098*"boy" + 0.075*"dies" + 0.018*"attempted" + 0.017*"new" + 0.015*"wheelchair" + 0.015*"tennis" + 0.015*"vehicle" + 0.014*"british" + 0.012*"news" + 0.012*"women"'),
 (1,
  '0.024*"murder" + 0.022*"world" + 0.018*"police" + 0.015*"victim" + 0.012*"highest" + 0.012*"rate" + 0.012*"crime" + 0.010*"trial" + 0.010*"shrien" + 0.008*"african"'),
 (2,
  '0.028*"british" + 0.024*"victim" + 0.019*"saves" + 0.017*"dead" + 0.015*"flamethrowers" + 0.015*"lucky" + 0.014*"alive" + 0.013*"car" + 0.013*"girlfriends" + 0.010*"legal"')]

In [63]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.043*"victim" + 0.027*"saves" + 0.024*"alive" + 0.023*"lucky" + 0.018*"girlfriends" + 0.013*"children" + 0.012*"flamethrowers" + 0.011*"huffingtonpost" + 0.011*"news" + 0.011*"tweet"'),
 (1,
  '0.030*"british" + 0.020*"women" + 0.020*"new" + 0.019*"murder" + 0.018*"motorists" + 0.018*"world" + 0.018*"naked" + 0.018*"method" + 0.017*"photos" + 0.017*"dead"'),
 (2,
  '0.135*"boy" + 0.103*"dies" + 0.021*"vehicle" + 0.012*"police" + 0.010*"news" + 0.009*"murder" + 0.007*"suspect" + 0.007*"violent" + 0.006*"horror" + 0.006*"attempted"'),
 (3,
  '0.027*"british" + 0.025*"wheelchair" + 0.025*"tennis" + 0.025*"attempted" + 0.016*"dead" + 0.014*"man" + 0.014*"victim" + 0.013*"champion" + 0.013*"newlywed" + 0.011*"news"')]